In [1]:
from IPython.display import display, HTML
from datetime import datetime

import json as json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv("raw_data.csv")
df = df.drop(columns=["Sorszám", "Alapbetegségek"])
df.drop(df.index[0:215],0,inplace=True)
df = df.reindex(index=df.index[::-1])
df.reset_index(drop=True, inplace=True)
df.tail(5)

C:\Users\bornalid\AppData\Local\Temp/ipykernel_2064/2552302721.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.index[0:215],0,inplace=True)


,Nem,Kor
32166,Nõ,89
32167,Nõ,78
32168,Férfi,70
32169,Nõ,85
32170,Férfi,70


In [3]:
read_file = pd.DataFrame(pd.read_excel("dates.xlsx"))
read_file.to_csv ("dates.csv",
                  index = None,
                  header=True)
df1 = pd.read_csv("dates.csv")
df1 = pd.read_csv("dates.csv", usecols=['Dátum', 'Elhunytak', 'New Deaths'])
df1.drop(df1.index[0:12],0,inplace=True)
df1['Elhunytak'] = df1['Elhunytak'].fillna(0)
df1['New Deaths'] = df1['New Deaths'].fillna(0)
df1.reset_index(drop=True, inplace=True)
print(df1.head())
len(df1)

FileNotFoundError: [Errno 2] No such file or directory: 'dates.xlsx'

In [ ]:
repeat = df1["New Deaths"]
data = pd.DataFrame(np.repeat(df1.values, repeat,axis=0), columns= df1.columns)
data.head()

In [ ]:
new_data = pd.merge(data, df, left_index=True, right_index=True)
new_data = new_data.drop(columns=['Elhunytak', 'New Deaths'])
new_data["Dátum"]=pd.to_datetime(new_data["Dátum"])
new_data.set_index("Dátum", inplace=True)
print(new_data.head())
len(new_data)

In [ ]:
df1["Dátum"]=pd.to_datetime(df1["Dátum"])
df1.set_index("Dátum", inplace=True)
data_with_dates=pd.merge(df1,new_data, how='inner', left_index=True, right_index=True)
data_with_dates.to_csv("data_with_dates.csv")
data_with_dates.reset_index(level=0, inplace=True)

In [ ]:
data_with_dates.head(10)

In [ ]:
bins= pd.DataFrame(pd.cut(x=data_with_dates['Kor'], bins=[0, 9, 19, 29, 39, 49,59,69, 79, 89,99, 109]))
bins.rename(columns={'Kor' :'Age'},inplace=True)
bins

In [ ]:
data_with_dates=pd.concat((data_with_dates,bins), axis = 1)
data_with_dates.drop(columns=["Elhunytak","New Deaths"], inplace = True)
data_with_dates.head(20)

In [ ]:
rep = pd.DataFrame(data_with_dates.groupby(pd.Grouper(freq='W', key='Dátum'))['Nem'].count())
rep.reset_index(inplace=True)
rep1 = rep["Nem"]
mydata = pd.DataFrame(np.repeat(rep.values, rep1,axis=0), columns= rep.columns)
mydata["Date"] = mydata["Dátum"] 
mydata["Count"] = mydata["Nem"]
mydata.drop(columns=["Dátum", "Nem"], inplace = True)
mydata.head()

In [ ]:
date_age=pd.merge(mydata, data_with_dates, how='inner', left_index=True, right_index=True)
date_age.head()
date_age.drop(columns=["Dátum"], inplace = True)
date_age["Date"] = date_age["Date"].dt.date
date_age.drop(columns=["Count", "Nem", "Kor" ], inplace = True)

date_age.head(10)

In [ ]:
aggregated_data = date_age.groupby( list(date_age))[['Date']].count()
aggregated_data = aggregated_data.unstack(level = 0)
aggregated_data.columns = aggregated_data.columns.droplevel()
aggregated_data = aggregated_data.reindex(index=aggregated_data.index[::-1])
aggregated_data.to_csv("aggregated_data.csv")
#aggregated_data.head()

In [ ]:
ax = sns.heatmap(aggregated_data)

In [ ]:
weekly=aggregated_data.columns

In [ ]:
rows=aggregated_data.index
rows=rows.categories[:10]
rows=rows[::-1]

In [ ]:
table=aggregated_data.to_numpy()
new_table=table[1:,:]

In [ ]:
file = open("populacio.json")
population = json.load(file)
dict_to_list = list(population.items())
new_list= pd.DataFrame(dict_to_list,columns=['Age','popln'])
new_list["Age"]=new_list["Age"].astype(int)
new_list.head()

In [ ]:
new_list['Age']= pd.DataFrame(pd.cut(x=new_list['Age'], bins=[-0.1, 9, 19, 29, 39, 49,59,69, 79, 89,99, 109]))
aggregated_list=new_list.groupby(['Age'])['popln'].agg('sum')

In [ ]:
table1=aggregated_list.to_numpy()
table1=table1[::-1]
table1=table1.reshape(len(table1), 1)

In [ ]:
table2=new_table/table1

In [ ]:
aggregated_data_ratio=pd.DataFrame(table2,columns=weekly, index=rows)
aggregated_data_ratio.to_csv("aggregated_data_ratio.csv")
#aggregated_data_ratio

In [ ]:
axx = sns.heatmap(aggregated_data_ratio)

In [ ]:
df["Nem"].replace({"férfi": "Férfi", "nõ": "Nõ", "Nő": "Nõ"}, inplace=True)
#df.head(20)

In [ ]:
df= df.groupby( list(df))[['Nem']].count()
df = df.unstack(level = 0)
df.reset_index(level=0, inplace=True)
#df

In [ ]:
df.columns = df.columns.droplevel()
df.rename(columns = {"" : "Kor"}, inplace=True )
df['Férfi'] =df['Férfi'].fillna(0)
df['Nõ'] =df['Nõ'].fillna(0)
#df

In [ ]:
df["Férfi"]=-df["Férfi"]

In [ ]:
plt.barh(df["Kor"], df[ "Férfi"], color = 'r', lw=0, )
plt.barh(df["Kor"], df[ "Nõ"], color = 'b',  lw=0, )
plt.title("Pyramid Age Distribution of Genders", fontsize=22)
plt.xlabel("Male/Female")
plt.show()